# 문장(시계열 수치)입력 이진 분류모델 (다층퍼셉트론 신경망)
- 문장을 입력해서 이진분류하는 모델 
- 문장을 시계열 열수치로 인코딩하는 방법과 여러가지 이진 분류 모델 사용
- 문장 혹은 시계열 수치로 양성/음성을 분류하거나 이베늩 발생 유무를 감지하는 문제 해결 

### 데이터 셋 준비
* IMDB에서 제공하는 영화 리뷰 데이터셋을 이용
* 데이터셋은 훈련셋 25,000개, 시험셋 25,000개의 샘플을 제공
* 라벨은 1과 0으로 좋아요/싫어요로 지정
* 케라스에서 제공하는 imdb의 load_data() 함수을 이용하면 데이터셋을 쉽게 얻을 수 있음. 
* 데이터셋은 이미 정수로 인코딩되어 있으며, 정수값은 단어의 빈도수를 나타냄
* 모든 단어를 고려할 수 없으므로 빈도수가 높은 단어를 위주로 데이터셋을 생성
* 20,000번째로 많이 사용하는 단어까지만 데이터셋으로 만들고 싶다면, num_words 인자에 20000이라고 지정하면 됨.

In [6]:
from tensorflow.keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)
print(len(x_train))
print(x_train[0])

25000
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


* 총 25000개의 샘플
* 각 샘플은 영화 리뷰 한 건을 의미
* 단어의 인덱스로 구성되어 있습니다. 
* num_words=20000’으로 지정했기 때문에 빈도수가 20,000을 넘는 단어는 보이지가 않음
* 훈련셋 25,000개를 다시 훈련셋 20,000개와 검증셋 5,000개로 분리

In [8]:
x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

* 모델의 입력으로 사용하려면 고정된 길이로 만들어야 함
* 케라스에서 제공되는 전처리 함수인 sequence의 pad_sequences() 함수를 사용
    * 문장의 길이를 maxlen 인자로 맞춰줍니다
    * 예를 들어 200으로 지정했다면 200보다 짧은 문장은 0으로 채워서 200단어로 맞춰주고 200보다 긴 문장은 200단어까지만 잘라냅니다.
    * (num_samples, num_timesteps)으로 2차원의 numpy 배열로 만들어줍니다. 
    * maxlen을 200으로 지정하였다면, num_timesteps도 200이 됩니다.

In [10]:
from tensorflow.keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=200)
x_val = sequence.pad_sequences(x_val, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)

### 다중퍼셉트론 신경망

* 임베딩(Embedding) 레이어
    * 첫번째 인자(input_dim) : 단어 사전의 크기를 말하며 총 20,000개의 단어 종류가 있다는 의미입니다. 
        이 값은 앞서 imdb.load_data() 함수의 num_words 인자값과 동일해야 합니다.
    * 두번째 인자(output_dim) : 단어를 인코딩 한 후 나오는 벡터 크기 입니다. 
        이 값이 128이라면 단어를 128차원의 의미론적 기하공간에 나타낸다는 의미입니다. 
        단순하게 빈도수만으로 단어를 표시한다면, 10과 11은 빈도수는 비슷하지만 단어로 볼 때는 전혀 다른 의미를 가지고 있습니다. 
        하지만 의미론적 기하공간에서는 거리가 가까운 두 단어는 의미도 유사합니다. 
        즉 임베딩 레이어는 입력되는 단어를 의미론적으로 잘 설계된 공간에 위치시켜 벡터로 수치화 시킨다고 볼 수 있습니다.
    * input_length : 단어의 수 즉 문장의 길이를 나타냅니다. 
        임베딩 레이어의 출력 크기는 샘플 수 * output_dim * input_lenth가 됩니다. 
        임베딩 레이어 다음에 Flatten 레이어가 온다면 반드시 input_lenth를 지정해야 합니다. 
        플래튼 레이어인 경우 입력 크기가 알아야 이를 1차원으로 만들어서 Dense 레이어에 전달할 수 있기 때문입니다.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Flatten

model = Sequential()
model.add(Embedding(20000, 128, input_length=200))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### 순환 신경망 모델 
* 문장을 단어들의 시퀀스로 간주하고 순환(LSTM) 레이어의 입력으로 구성한 모델입니다. 
* 임베딩 레이어 다음에 LSTM 레이어가 오는 경우에는 임베딩 레이어에 input_length 인자를 따로 설정할 필요는 없습니다. 
* 입력 문장의 길이에 따라 input_length가 자동으로 정해지고, 이것이 LSTM 레이어에는 timesteps으로 입력되기 때문입니다. 
* 블록으로 표현한다면 예제에서는 문장의 길이가 200 단어이므로, LSTM 블록 200개가 이어져있다고 생각하면 됩니다.
* LSTM에 입력되는 타임스텝은 Embedding 출력 타임스텝으로 200이고, 특징 크기는 Embedding에서 인코딩된 128입니다.

In [14]:
from tensorflow.keras.layers import LSTM
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

### 컨볼루션 신경망 모델
* 문장 해석에 컨볼루션(Conv1D) 레이어를 이용한 모델입니다. 
    * 컨볼루션(Conv1D) 레이어는 위치에 상관없이 지역적인 특징을 잘 뽑아냅니다. 
* 이 레이어를 문장에 적용한다면 주요 단어가 문장 앞 혹은 문장 뒤에 있더라도 놓치지 않고 전후 문맥을 보면서 특징을 잘 뽑아낼 수 있습니다. 
* 글로벌 맥스풀링(GlobalMaxPooling1D) 레이어는 컨볼루션 레이어가 문장을 훑어가면서 나온 특징벡터들 중 가장 큰 벡터를 골라줍니다. 
    * 즉 문맥을 보면서 주요 특징을 뽑아내고, 그 중 가장 두드러지는 특징을 고르는 것입니다.

In [15]:
from tensorflow.keras.layers import Dropout, Conv1D, GlobalMaxPooling1D
model = Sequential()
model.add(Embedding(20000, 128, input_length=200))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

### 순환 컨볼루션 신경망 모델
* 컨볼루션 레이어에서 나온 특징벡터들을 맥스풀링(MaxPooling1D)를 통해 1/4로 줄여준 다음 LSTM의 입력으로 넣어주는 모델입니다. 
    * 이때 맥스풀링은 특징벡터 크기를 줄여주는 것이 아니라 특징벡터 수를 줄여줍니다. 
    * 즉 200개 단어가 컨볼루션 레이어를 통과하면 256 크기를 갖는 특징벡터가 198개가 생성되고, 맥스풀링은 특징벡터 198개 중 49개를 골라줍니다. 
    * 따라서 LSTM 레이어의 timesteps는 49개가 됩니다. 
* 참고로 input_dim은 그대로 256입니다.
* LSTM에 입력되는 타임스텝은 49, 속성은 256입니다. 
    * 타임스텝이 49인 이유는 Conv1D에서 200단어를 받아 198개를 반환하고, 이를 다시 MaxPooling1D에 의해 1/4배로 줄어들어 49가 된 것입니다.
    * 속성이 256인 이유는 Conv1D가 Embedding 출력인 128 벡터를 입력받아 256으로 반환되기 때문입니다.

In [18]:
from tensorflow.keras.layers import MaxPooling1D
model = Sequential()
model.add(Embedding(20000, 128, input_length=200))
model.add(Dropout(0.2))
model.add(Conv1D(256,
                 3,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))